In [1]:
import pandas as pd
import os
import re
import requests
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import time
import tqdm
from pyvirtualdisplay import Display
import json

In [108]:
display = Display(visible=0, size=(1920,1080))
display.start()

In [109]:
path = '/home/harong/chromedriver'
driver = webdriver.Chrome(path)

In [110]:
# 여기서 무조건 로그인 해야함. 
driver.get('https://nid.naver.com/nidlogin.login?svctype=262144&url=http://naver.com/')

In [111]:
driver.get('https://cafe.naver.com/mbticafe')

In [112]:
driver.find_element_by_xpath('//*[@id="menuLink0"]').click()

In [113]:
def crawl_mbti_data() :
    tmp_li = dict()
    cnt = 1
    for i in tqdm.tqdm(range(602268,0,-1)) : 
        addr = "https://cafe.naver.com/mbticafe/ArticleRead.nhn?clubid=11856775&boardtype=L&articleid="+str(i)+"&referrerAllArticles=true"
        driver.get(addr)
        try : 
            driver.switch_to.frame('cafe_main')
            html = driver.page_source
            soup = bs(html, "html.parser")
        except : 
            continue
        try : 
            category = soup.findAll("a", {"class": "m-tcol-c"})[3].text
            title = soup.find("span",{"class":"b m-tcol-c"}).text
            writer = soup.find("a",{"m-tcol-c b"}).text
            article = soup.find("div", {"class": "tbody m-tcol-c"}).text.replace('\u200b', ' ').replace('\n','').strip()

            tmp_li[i] = {'writer':writer,'category':category,'title':title,'article':article}
        except : 
            continue
        if len(tmp_li) % 100 == 0 :
            with open(f'/home/harong/mbti_project/crawl_data/crawl_res_{cnt}.json','w',encoding='utf-8') as f:
                json.dump(tmp_li,f,ensure_ascii=False)
            f.close()
            cnt += 1
            tmp_li = dict()

## 데이터 분석 및 전처리

In [2]:
with open('crawl_data/crawl_res_1.json', 'r') as f:
    data = json.load(f)

In [5]:
data

{'602266': {'writer': '에에엔팁(0721****)',
  'category': 'INFP ♧ ENFP',
  'title': '엔프피 연락',
  'article': '서로 알게 된지 일주일 정도 지난 엔프피녀가 있는데 일주일 내내 연락을 매일 하고 있거든요. 답장 텀도 항상 30분 이내길래 좀 소름 돋기도 하고 가끔 두시간 넘어가면 과외하러 간다, 피곤해서 낮잠잤다, 갑자기 친한 친구 만나서 얘기하다 왔다 이렇게 꼭 말을 해주더라구요. 근데 워낙 엔프피들이 사람을 좋아하는 성격이라서 호감이 있어서 그런건지 아님 그냥 친한 친구로 봐서 그런건지 궁금합니다. 저는 개인적으로 호감 있는 사람에게는 엄청 칼답이지만 친한 친구들은 거의 뭐...ㅋㅋ 제가 심심할때 주로 답하고 연락하는 스타일이라 엔프피들은 어떤지 궁금합니다. 저는 솔직히 그 친구 한테 호감이 있는데 모두에게 잘해주는 엔프피 특성상 조금 조심스럽네요ㅠㅎ리액션도 엄청 잘해주고 이모티콘도 많이 쓰는데 그건 원래 엔프피만의 특징인거 같기도 하고...'},
 '602264': {'writer': '커피 그마안(puln****)',
  'category': '신변 잡기',
  'title': '신라면 맛 변했나요?',
  'article': '계속 진라면 매운맛 먹다가오랜만에 신라면 먹었는데엄청 느끼하고 안 매워요.. 맛이 변한 건지 제 입맛이 변한 건지...'},
 '602263': {'writer': '초코나무(wpzm****)',
  'category': '고민, 연애, 진로 상담',
  'title': '여자 isfj / 남자intj 연애 ',
  'article': '공감 능력이 탑쪽인 여자와공감 능력이 그냥저냥인 남자와의 연애 4개월 쫌 지났네요  거기에 장거리 (약 120키로) 연애입니다   남자는컨디션이나 상황이 되면평일에도 만나러 오기도 하고 (1-2시간 볼려고 왕복 3-4시간 거리 옵니다. 여자보다 이동이 자유롭습니다. 차는 둘다 있구요)주말에도 하루정도는 서로에게 오롯이 집중

In [ ]:
mbti_json = {"엔티제": "ENTJ",
             "엔팁": "ENTP",
             "인티제": "INTJ",
             "인팁": "INTP",
             "엔프피": "ENFP",
             "엔프제": "ENFJ",
             "인프피": "INFP",
             "인프제": "INFJ",
             "엣티제": "ESTJ",
             "엣프제": "ESFJ",
             "잇티제": "ISTJ",
             "잇프제": "ISFJ",
             "엣팁": "ESTP",
             "엣픕": "ESFP",
             "엣프피": "ESFP",
             "잇팁": "ISTP",
             "잇픕": "ISFP",
             "잇프피": "ISFP"
             }